In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Call Api on File Upload

Attributes
----------
subscriptions
events
notifications
luminesce
"""

toggle_code("Toggle Docstring")

# Call API on File Upload

This notebook will detail how to create a process in which actions are triggered on the upload of a new file.

For this example, we will manually drop an auto-load-transactions.xlsx file into drive, and this will trigger an api call which will run a Luminesce query to create a portfolio, create instruments and upsert the transactions into that portfolio. This is a very simple example, but it could be extended further by triggering a job which could contain references to any number of apis.

The steps are as follows:
1. Authorise LUSID account
2. Create subscription to FileCreated event
3. Create webhook notification to be triggered by subscription
    - This notification will call an API
4. Trigger the event

### Setup Lusid

Import modules and define our api factories.

In [4]:
import os
import json
import pandas as pd

# import lusid specific packages
import lusid.models as models
import lusid.utilities as utilities
import lusid
from fbnsdkutilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid_notifications as ln
import lusid_drive

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_url = os.getenv("FBN_LUSID_API_URL")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

notifications_factory = ApiClientFactory(ln, token=RefreshingToken())

portfolios_factory = utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path
)

In [ ]:
# notifications APIs
subscriptions_api = notifications_factory.build(ln.api.SubscriptionsApi)
notifications_api = notifications_factory.build(ln.api.NotificationsApi)
events_api = notifications_factory.build(ln.api.EventsApi)
event_types_api = notifications_factory.build(ln.api.EventTypesApi)

# portfolios API
portfolios_api = portfolios_factory.build(lusid.api.TransactionPortfoliosApi)

# drive API
configuration = lusid_drive.Configuration(
    host=f'{portfolios_api.api_client.configuration.host[:-4]}/drive'
)
configuration.access_token = os.getenv("FBN_ACCESS_TOKEN")

with lusid_drive.ApiClient(configuration) as api_client:
    drive_api = lusid_drive.FilesApi(api_client)

### 1. Authorise LUSID account

To authorise your LUSID account, follow the instruction at [KA-01735](https://support.lusid.com/knowledgebase/article/KA-01735/). This will allow the webhook to execute an API request on your behalf. You only need to do this once for each Lusid account.


### 2. Create a subscription

Here we are creating a subscription to a `FileCreated` event, with a matching filter set to look for our `auto-load-transactions.xlsx` file.

In [5]:
# create subscription to file creation event
def create_file_subscription(
        scope: str,
        code: str,
        name: str,
        matching_filter: str,
        description: str = None
):
    try:
        resp = subscriptions_api.create_subscription(
            create_subscription=ln.models.CreateSubscription(
                id=models.ResourceId(
                    scope=scope,
                    code=code
                ),
                display_name=name,
                description=description if description else name,
                status="Active",
                matching_pattern=ln.models.MatchingPattern(
                    event_type="FileCreated",
                    filter=matching_filter
                )
            )
        )
        print(f"Subscription created for {code}.")
        return resp
    except ln.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 711: # 'SubscriptionAlreadyExists'
            raise e

create_file_subscription(
    scope="file_load",
    code="file_load",
    name="file_load",
    matching_filter="FileName eq 'auto-load-transactions.xlsx'",
)

Subscription created for file_load.


{'created_at': datetime.datetime(2023, 2, 7, 10, 12, 27, 756852, tzinfo=tzutc()),
 'created_by': '00uieujafoYdmkDSx2p7',
 'description': 'file_load',
 'display_name': 'file_load',
 'id': {'code': 'file_load', 'scope': 'file_load'},
 'last_modified_at': datetime.datetime(2023, 2, 7, 10, 12, 27, 756852, tzinfo=tzutc()),
 'last_modified_by': '00uieujafoYdmkDSx2p7',
 'matching_pattern': {'event_type': 'FileCreated',
                      'filter': "FileName eq 'auto-load-transactions.xlsx'"},
 'status': 'Active'}

### 3. Create a webhook notification

First, define the SQL Luminesce query to run. This query will unpack our data file from the drive and then create a portfolio, instruments and transactions.


In [6]:
sql = """
-- Extract transaction data from LUSID Drive

@txn_data =
use Drive.Excel
--file=/lusid-examples/auto-load-transactions.xlsx
enduse;

-- Set variables for the portfolio's scope and code

@@portfolio_scope = select 'auto-load-demo';
@@portfolio_code = select 'uk-equity';
@@portfolio_name = select 'UK EQUITY';

-- Define the portfolio data

@create_portfolio =
select 'Transaction' as PortfolioType,
@@portfolio_scope as PortfolioScope,
@@portfolio_code as PortfolioCode,
@@portfolio_name as DisplayName,
'' as Description,
#2000-01-01# as Created,
''as SubHoldingKeys,
'GBP' as BaseCurrency;

-- Upload the portfolio into LUSID

@response_create_portfolio =
select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

-- Get instrument data

@equity_instruments =
select
Name as DisplayName,
ISIN as Isin,
ClientInternal as ClientInternal,
SEDOL as Sedol,
'GBP' as DomCcy
from @txn_data;

-- Upload the transformed data into LUSID

@response = select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments;

-- --Transform data using SQL

@transactions =
select
@@portfolio_scope as PortfolioScope,
@@portfolio_code as PortfolioCode,
t.TransactionID as TxnId,
t.Type as Type,
t.TransactionDate as TransactionDate,
t.SettlementDate as SettlementDate,
t.Units as Units,
t.Price as TradePrice,
t.TotalConsideration as TotalConsideration,
t.Currency as SettlementCurrency,
t.ClientInternal as ClientInternal,
r.LusidInstrumentId as LusidInstrumentId
from @txn_data t
inner join @response r
where t.ClientInternal = r.ClientInternal;

-- Upload the transformed data into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @transactions;
"""

Now, define the webhook notification to run our SQL query.

In [15]:
def create_webhook_notification(
        scope: str,
        code: str,
        sql: str,
):
    # this can be changed to call any api required
    create_webhook_notification = {
        "description": "Upsert transaction into new portfolio",
        "httpMethod": "Put",
        "url": "/honeycomb/api/Sql/csv",
        "authenticationType": "Lusid",
        "contentType": "PlainText",
        "content": sql
    }

    try:
        response = notifications_api.create_webhook_notification(
            scope=scope,
            code=code,
            create_webhook_notification=create_webhook_notification
        )
        return f"Notification successfully created with ID : {response.id}"
    except ln.ApiException as e:
        return e.body

# delete notifications to allow updates
notifications = notifications_api.list_notifications(scope="file_load", code="file_load").values
for notification in notifications:
    notifications_api.delete_notification(
        scope="file_load",
        code="file_load",
        id=notification.id
    )

create_webhook_notification(
    scope="file_load",  # scope and code of subscription
    code="file_load",
    sql=sql
)

'Notification successfully created with ID : 7bf05c94-2d74-4da7-9f68-3930129c763e'

### 4. Trigger the event

You can trigger the event by dropping the `auto-load-transactions.xlsx` file into drive. This file is located within the data folder. You can verify the process has been successful by checking your portfolios for `uk-equity` within the `auto-load-demo` scope.

Alternatively you can run the following cells.

The following cell verifies that the portfolio doesn't exist

In [25]:
try:
    portfolios_api.get_transactions(scope='auto-load-demo', code='uk-equity').values
except lusid.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 109: # 'PortfolioNotFound'
        raise e
    else:
        print(detail['title'])

Portfolio not found


Next, upload the file to drive. This will create a `FileCreated` event meaning that our event subscription will get triggered. This will kick off our webhook notification which will in turn call the Luminesce SQL API endpoint, with our SQL script as its payload. This SQL will run and begin the creation of our portfolio, instruments and transactions.

In [37]:
file_name = 'auto-load-transactions.xlsx'
drive_path = '/lusid-examples'

with open('./data/auto-load-transactions.xlsx', 'rb') as data:
    try:
        response = drive_api.create_file(
            x_lusid_drive_filename=file_name,
            x_lusid_drive_path=drive_path,
            content_length=os.stat('./data/auto-load-transactions.xlsx').st_size,
            body=data.read()
        )
        href = f"https://{portfolios_api.api_client.configuration.host[:-4]}/app/data-management/drive/{response.id}?type=file"
    except lusid_drive.rest.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 671: # 'FileAlreadyExists'
            raise e

print("File created at:", href)

File created at: https://fbn-dan.lusid.com/app/data-management/drive/7d3c87a4-cbe3-47da-80e8-1b74626323d9?type=file


Now check that the portfolio exists and contains the expected transactions

In [49]:
try:
    values = portfolios_api.get_transactions(scope='auto-load-demo', code='uk-equity').values
    portfolio_txns = pd.DataFrame(
        {
            'transaction_id': [i.transaction_id for i in values],
            'type': [i.type for i in values],
            'units': [i.units for i in values],
            'price': [i.transaction_price.price for i in values],
            'amount': [i.total_consideration.amount for i in values],
            'transaction_date': [i.transaction_date for i in values],
            'settlement_date': [i.settlement_date for i in values],
            'transaction_currency': [i.transaction_currency for i in values],
        }
    )
except lusid.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 109: # 'PortfolioNotFound'
        raise e

portfolio_txns

,transaction_id,type,units,price,amount,transaction_date,settlement_date,transaction_currency
0,txn-0006,Buy,100.0,2203.79,220379.0,2022-01-13 00:00:00+00:00,2022-01-16 00:00:00+00:00,GBP
1,txn-0003,Buy,100.0,45.31,4531.0,2022-02-04 00:00:00+00:00,2022-02-06 00:00:00+00:00,GBP
2,txn-0001,Buy,100.0,161.02,16102.0,2022-03-01 00:00:00+00:00,2022-03-04 00:00:00+00:00,GBP
3,txn-0005,Sell,100.0,1136.50,113650.0,2022-03-03 00:00:00+00:00,2022-03-05 00:00:00+00:00,GBP
4,txn-0002,Buy,100.0,1453.20,145320.0,2022-05-03 00:00:00+00:00,2022-05-07 00:00:00+00:00,GBP
5,txn-0004,Buy,100.0,286.40,28640.0,2022-05-03 00:00:00+00:00,2022-05-06 00:00:00+00:00,GBP
6,txn-0007,Sell,100.0,726.40,72640.0,2022-07-24 00:00:00+00:00,2022-07-27 00:00:00+00:00,USD
7,txn-0008,Sell,100.0,1214.00,121400.0,2022-07-31 00:00:00+00:00,2022-08-02 00:00:00+00:00,USD
8,txn-0009,Buy,100.0,263.70,26370.0,2022-08-09 00:00:00+00:00,2022-08-12 00:00:00+00:00,GBP
